# Analyze different architectures for **License plate detection** using **Yolo**

We define the following pipeline:
- A yolo model pretrained on *COCO 2017* dataset and fine-tuned using an additional dataset for detection of cars, trucks, pedestrians and traffic lights (this model will be used to extract region of interest (ROI) from a frame);
- A yolo model pretrained on *COCO 2017* dataset and fine-tuned using a new dataset for license plate detection;
- **maybe** a model using *particle filters* used to track identified license plate from different frames without apply tracking on consecutive frames;
- A convolutional neural network used for character recognition from license plate.

### 1. Vehicle detection model

First of all, we must install **ultralytics** package and load a nano version of **Yolov8** model, pretrained on *COCO 2017* dataset.

In [1]:
%pip install ultralytics
from ultralytics import YOLO
import torch

print(torch.cuda.is_available())

model = YOLO('yolov8s.pt')
model.cuda()

  Obtaining dependency information for ultralytics from https://files.pythonhosted.org/packages/02/29/7e9d2905d25a7727f7cf56fbf2e79cc36d60481cf3249143eee1b7217f29/ultralytics-8.1.29-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 3.7 MB/s eta 0:00:00
  Obtaining dependency information for thop>=0.1.1 from https://files.pythonhosted.org/packages/bb/0f/72beeab4ff5221dc47127c80f8834b4bcd0cb36f6ba91c0b1d04a1233403/thop-0.1.1.post2209072238-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 721.3/721.3 kB 28.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
True


YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(96, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_s

Then we train the model using this configuration:
* *epochs* = 100 (number of epochs for training)
* *imgsz* = 640 (set height of image to 640px)
* *batch* = -1 (allow package to choose optimal batch size)
* *pretrained* = True (fine-tune model using pretrained weights)

In [ ]:
# Train the model
results = model.train(data='/kaggle/input/selfdrive-object-detection/data.yaml', epochs=100, imgsz=640, batch=-1, plots=True, pretrained=True)

engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/kaggle/input/selfdrive-object-detection/data.yaml, epochs=100, time=None, patience=100, batch=-1, imgsz=640, save=True, save_period=-1, cache=False, device=cuda:0, workers=8, project=None, name=train4, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_width=None, format=torchscript, keras=Fals

100%|██████████| 755k/755k [00:00<00:00, 37.9MB/s]
2024-03-18 12:43:44,247	INFO util.py:129 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-03-18 12:43:44,748	INFO util.py:129 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Overriding model.yaml nc=80 with nc=5

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytics

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     26/100      10.3G      1.291     0.7217      1.037        234        640: 100%|██████████| 345/345 [03:37<00:00,  1.59it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 29/29 [00:13<00:00,  2.23it/s]

                   all       2183      14972      0.765      0.655      0.706      0.386



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     27/100      10.3G      1.287     0.7182      1.035        393        640:  94%|█████████▍| 324/345 [03:24<00:13,  1.58it/s]

**Maybe** we give up this method and use a trained model from Roboflow :). Let's load this model and view results on test data (No, not a good idea :) ).

So, we are back and we will upload the model weights to roboflow dataset.

In [5]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="IUxYJzulWVkv30nEW5J0")
project = rf.workspace("selfdriving-traffic-detection").project("self-driving-traffic-detection")
version = project.version(4)

version.deploy("yolov8", "/kaggle/working/runs/detect/train3")

loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.1.29, to fix: `pip install ultralytics==8.0.196`


KeyboardInterrupt: Interrupted by user

Next, we try to test model using videos with different scenarios. Here are some examples: different car speeds (30km/h, 50km/h, 70km/h) an light conditions (day, sunset, foggy).

In [2]:
%pip install supervision
import supervision as sv
import numpy as np
from ultralytics import YOLO
import cv2

VIDEO_PATH = "/kaggle/input/cristi/cristi-2.mp4"
TARGET_PATH = "/kaggle/working/inference/cristi-2-test.mp4"
model = YOLO("/kaggle/working/runs/detect/train3/weights/best.pt")


class VideoInference(object):
    def __init__(self, video_path: str, model):
        self._video_path = video_path
        self._model = model
        self._byte_tracker = sv.ByteTrack()
        
    def adjust_brightness(self, image, brightness_factor):
        # Convert image to HSV color space
        hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        # Scale the V (brightness) channel
        hsv[:,:,2] = np.clip(hsv[:,:,2] * brightness_factor, 0, 255)
        # Convert the image back to BGR color space
        return cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
        
    def run(self, target_path: str):
        annotator = sv.BoxAnnotator()
        video_info = sv.VideoInfo.from_video_path(self._video_path)
        
        def callback(frame: np.ndarray, index: int) -> np.ndarray:
            # Detect objects in the frame
            results = model(frame, conf=0.3)[0]
            detections = sv.Detections.from_ultralytics(results)
            detections = self._byte_tracker.update_with_detections(detections)

            # Adjust brightness of the entire frame (background)
            frame = self.adjust_brightness(frame, brightness_factor=0.5)  # Decrease brightness

            # Create a mask for detected objects
            mask = np.zeros_like(frame[:, :, 0], dtype=np.uint8)
            for detection in detections:
                x1, y1, x2, y2 = detection[0]
                # print(x1, y1, x2, y2)
                cv2.rectangle(mask, (int(x1), int(y1)), (int(x2), int(y2)), (255, 255, 255), -1)

            # Adjust brightness of the detected objects (contours)
            contour_brightness_factor = 1.5  # Increase brightness
            frame_contours = self.adjust_brightness(frame, brightness_factor=contour_brightness_factor)

            # Combine the frames with adjusted brightness for background and contours
            frame_with_contours = cv2.bitwise_and(frame, frame, mask=cv2.bitwise_not(mask))
            frame_with_contours += cv2.bitwise_and(frame_contours, frame_contours, mask=mask)

            labels = [
                f"#{tracker_id} {model.model.names[class_id]} {confidence:0.2f}"
               for _, _, confidence, class_id, tracker_id, _
              in detections
            ]
            
            return annotator.annotate(scene=frame_with_contours.copy(), detections=detections, labels=labels)

        sv.process_video(source_path=VIDEO_PATH, target_path=target_path, callback=callback)

video_inference = VideoInference(VIDEO_PATH, model)
video_inference.run(TARGET_PATH)

  Obtaining dependency information for supervision from https://files.pythonhosted.org/packages/9c/8a/8e86cb06d6e10d532c54722977d2b06797026336729ce37a2d35504f23d6/supervision-0.19.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.1/97.1 kB 8.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


SupervisionWarnings: BoxAnnotator is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `BoundingBoxAnnotator` and `LabelAnnotator` instead



0: 384x640 1 car, 95.5ms
Speed: 9.8ms preprocess, 95.5ms inference, 197.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 6.5ms
Speed: 1.6ms preprocess, 6.5ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 6.9ms
Speed: 1.9ms preprocess, 6.9ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 6.5ms
Speed: 1.7ms preprocess, 6.5ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 6.5ms
Speed: 1.6ms preprocess, 6.5ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 6.6ms
Speed: 1.6ms preprocess, 6.6ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 6.7ms
Speed: 1.5ms preprocess, 6.7ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 6.7ms
Speed: 1.6ms preprocess, 6.7ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 7.1ms
Spe

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)




0: 384x640 2 cars, 6.6ms
Speed: 1.7ms preprocess, 6.6ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 6.5ms
Speed: 1.6ms preprocess, 6.5ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 6.7ms
Speed: 2.0ms preprocess, 6.7ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 6.6ms
Speed: 1.9ms preprocess, 6.6ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 7.1ms
Speed: 1.8ms preprocess, 7.1ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 7.5ms
Speed: 1.9ms preprocess, 7.5ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 7.5ms
Speed: 1.8ms preprocess, 7.5ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 7.4ms
Speed: 1.8ms preprocess, 7.4ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 7.0m

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



### 2. License plate detection model

Now we have **ultralytics** installed, so we only need to create a new model. This model will be trained using a dataset of around 20k annotated images with license plates and will be applied on ROI extracted using first model.

This model has already been trained so now we only load best obtained weights.

In [ ]:
from ultralytics import YOLO
import torch

print(torch.cuda.is_available())

model = YOLO("/kaggle/working/runs/detect/train9/weights/best.pt")
model.cuda()